In [1]:
__author__ = "Kexin Yu"
__version__ = "CS341, Stanford, Spring 2018"

In [2]:
import pandas as pd
df = pd.read_pickle('df_daily.pickle')

In [3]:
#df.dropna() # 2016-08-18 - 2018-04-03
mean_price_mat = df.dropna()[['BTC_mean', 'ETH_mean', 'LTC_mean']]
mean_price_mat.head()

,BTC_mean,ETH_mean,LTC_mean
time,,,
2016-08-18,575.137264,10.910598,3.620000
2016-08-19,574.886952,10.782314,3.750000
2016-08-21,581.963888,11.137447,4.000000
2016-08-22,582.617917,11.224444,3.990000
2016-08-23,583.219936,11.042448,4.136444


In [4]:
observed = mean_price_mat[['BTC_mean', 'ETH_mean', 'LTC_mean']].as_matrix()

col_sum = mean_price_mat.sum(axis=0).values # sum the cols
total_sum = mean_price_mat.values.sum()
mean_price_mat['rowsum'] = mean_price_mat.sum(axis=1) # sum the rows
row_sum = mean_price_mat['rowsum'].values

import numpy as np

row_sum = np.expand_dims(row_sum, axis=1)
col_sum = np.expand_dims(col_sum, axis=1)

expected = np.dot(row_sum, col_sum.T)/total_sum
#expected.shape
oe_reweighted = observed / expected
#oe_reweighted.shape #(593, 3)

In [5]:
oe_reweighted_price = pd.DataFrame(
    oe_reweighted, 
    columns=['BTC_oe_reweighted','ETH_oe_reweighted', 'LTC_oe_reweighted'])

indices = mean_price_mat.index.tolist()
indices = pd.Index(indices).set_names('time')
oe_reweighted_price.reset_index()
oe_reweighted_price = oe_reweighted_price.set_index([indices])
oe_reweighted_price.head()

,BTC_oe_reweighted,ETH_oe_reweighted,LTC_oe_reweighted
time,,,
2016-08-18,1.051968,0.311047,0.460205
2016-08-19,1.051954,0.307520,0.476933
2016-08-21,1.051203,0.313562,0.502184
2016-08-22,1.051098,0.315624,0.500316
2016-08-23,1.051188,0.310213,0.518188


In [6]:
df_new = df.join(oe_reweighted_price)

# reorder columns
cols = df_new.columns.tolist()
order_new = cols[:6] + [cols[-3]] + cols[6:12] + [cols[-2]] + cols[12:18] + [cols[-1]]
df_new = df_new[order_new]
df_new.tail().iloc[:,[3, 6, 10, 13, 17, 20]]

,BTC_mean,BTC_oe_reweighted,ETH_mean,ETH_oe_reweighted,LTC_mean,LTC_oe_reweighted
time,,,,,,
2018-03-30,6889.265958,1.005325,384.908007,0.875462,116.856743,1.185221
2018-03-31,7007.866653,1.003801,401.746444,0.896934,120.071590,1.195402
2018-04-01,6804.774833,1.005162,382.173000,0.879891,114.626632,1.176846
2018-04-02,6978.227958,1.005944,385.098333,0.865260,118.544951,1.187745
2018-04-03,7323.964000,1.006196,400.897891,0.858453,125.731609,1.200584


In [7]:
df_new.to_pickle('df_daily_2.pickle')